In [35]:
import math
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from statsbombpy import sb 
from mplsoccer import VerticalPitch

In [36]:
df = sb.events(match_id=3869685).sort_values(by=['index'])

In [37]:
team1 = df[df.team == 'Argentina']
team2 = df[df.team == 'France']

In [38]:
FRANCE_COLOR = '#002654'
ARGENTINA_COLOR = '#43A1D5'

ARGENTINA_TEAM_ID = 779
FRANCE_TEAM_ID = 771

In [39]:
df = df[df.period < 5] #odsortowanie wykonywanych serii rzutów karnych

In [44]:
def create_shotmap(team, ax):
    pitch = VerticalPitch(pitch_type='statsbomb', half=True)
    pitch.draw(ax=ax)
    
    #show only  shots and they are in a type column 
    team = team[team['type'] == 'Shot']
    #only columns which we need
    team = team[['team', 'player', 'location', 'shot_outcome', 'shot_statsbomb_xg', 'shot_type']]
    team[['x', 'y']] = team['location'].apply(pd.Series)
    
    #draw the shot map
    for x in team.to_dict(orient='records'):
        pitch.scatter(
        x=x['x'], y=x['y'], ax=ax, 
        s=500 * x['shot_statsbomb_xg'], 
        color='green' if x['shot_outcome'] == 'Goal' else 'white', 
        edgecolors='black', alpha=.7
        )
        

In [82]:
def create_passnetwork(team, ax):
    team[['x', 'y']] = team['location'].apply(pd.Series)
    
    tactics = team[~team['tactics'].isnull()][['tactics', 'team']]
    
    all_jersey_numbers = []
    for y in tactics.to_dict(orient='records'):
        all_jersey_numbers.append(pd.DataFrame([{'player_id':x['player']['id'], 'jersey_number':x['jersey_number']} for x in y['tactics']['lineup']]))
    jersey_number = pd.concat(all_jersey_numbers).drop_duplicates()
    
    team['newsecond']= 60 * team['minute'] + team['second']
    
    team.sort_values(by=['newsecond'])
    
    team['passer'] = team['player_id']
    
    passes_df = team.loc[(team['type'] == 'Pass')].copy()
    passes_df['outcome'] = passes_df['pass_outcome'].fillna('Successful')

    
    completions = passes_df.loc[(passes_df['pass_outcome'] == 'Successful')]
    
    
    #first substitution 
    sub_df = team.loc[(team['type'] == 'Substitution')]
    first_sub = sub_df['newsecond'].min()
    
    if first_sub <= (60 * 45):
        first_sub = 60 * 45
    
    completions = completions.loc[completions['newsecond'] < first_sub]
    
    #finding average locations 
    average_locs_and_count = completions.groupby('passer').agg({'x':['mean'], 'y': ['mean', 'count']})
    average_locs_and_count.columns = ['x', 'y', 'count']
    
    #number of passes between passer and recipient 
    passes_between = completions.groupby(['passer', 'recipient']).id.count().reset_index()
    passes_between.rename({'id': 'pass_count'}, axis='columns', inplace=True)
    
    passes_between = passes_between.merge(average_locs_and_count, left_on='passer', right_index=True)
    passes_between = passes_between.merge(
        average_locs_and_count, left_on='recipient', right_index=True, suffixes=['', '_end']
    )
    
    #minimum passes between players 
    passes_between = passes_between.loc[(passes_between['pass_count'] >= 4)]
    
    #plot arrows
    def pass_line_template(ax, x, y, end_x, end_y, line_color):
        ax.annotate(
            '', 
            xy=(end_y, end_x),
            xytext=(y, x),
            zorder=1,
            arrowprops=dict(arrowstyle='-|>', linewidth=4, color=line_color, alpha=.85)
        )
        
    def pass_line_template_shrink(ax, x, y, end_x, end_y, line_color, dist_delta=1.2):
        dist = math.hypot(end_x - x, end_y - y)
        angle = math.atan2(end_y-y, end_x-x)
        upd_x = x + (dist - dist_delta) * math.cos(angle)
        upd_y = y + (dist - dist_delta) * math.sin(angle)
        pass_line_template(ax, x, y, upd_x, upd_y, line_color=line_color)
        
    pitch = VerticalPitch(pitch_type='statsbomb')
    pitch.draw(ax=ax)
    
    for index, row in passes_between.iterrows():
        pass_line_template_shrink(ax, row['x'], row['y'], row['x_end'], row['y_end'], 'black')
        
    pitch.scatter(
        average_locs_and_count.x, average_locs_and_count.y, s=500, 
        color='#f0ece2', edgecolor='#010101', linewidth=2, alpha=1, ax=ax, zorder=2
    )
    
    for index, row in average_locs_and_count.iterrows():
        pitch.annotate(
            jersey_number[jersey_number['player_id'] == row.name]['jersey_number'].value[0],
            xy=(row.x, row.y),
            c='#132743',
            va='center',
            ha='center',
            size=10,
            fontweight='bold',
            ax=ax
        )    
    

In [107]:
def create_table(team1, team2):
    #droping when pass_outcome is unknown 
    team1 = team1[team1['pass_outcome'] != 'Unknown']
    team2 = team2[team2['pass_outcome'] != 'Unknown']
    
    #calculate stats
    #create dictioniary to store stats which were calculate
    stats = {
        'goals': {'team1': len(team1[team1['type'] == 'Goal']), 'team2': len(team2[team2['type'] == 'Goal'])},
        'xG': {'team1': round(team1['shot_statsbomb_xg'].sum(), 2), 'team2': round(team2['shot_statsbomb_xg'].sum(), 2)},
        'shots': {'team1': len(team1[team1['type'] == 'Shot']), 'team2': len(team2[team2['type'] == 'Shot'])},
        'shots_on_target': {
            'team1': len(team1[(team1['type'] == 'Shot') & (team1['shot_end_location'].str.len() > 2) & (team1['shot_outcome'] != 'Off T')]),
            'team2': len(team2[(team2['type'] == 'Shot') & (team2['shot_end_location'].str.len() > 2) & (team2['shot_outcome'] != 'Off T')])
        },
        'passes' : {'team1': len(team1[team1['type'] == 'Pass']), 'team2': len(team2[team2['type'] == 'Pass'])}, 
        'pass_completion': {
            'team1': round(len(team1[(team1['type'] == 'Pass') & (team1['pass_outcome'].isna())]) / len(team1[team1['type'] == 'Pass']) * 100, 2),
            'team2': round(len(team2[(team2['type'] == 'Pass') & (team2['pass_outcome'].isna())]) / len(team2[team2['type'] == 'Pass']) * 100, 2)
        }
    }
    stats_df = pd.DataFrame(stats).T
    
    column_labels = ['', '', '']
    
    table_vals = [
        [str(stats_df['team1']['xG']), 'xG', str(stats_df['team2']['xG'])],
        [str(stats_df['team1']['shots']), 'Shots', str(stats_df['team2']['shots'])],
        [str(stats_df['team1']['shots_on_target']), 'Shots on Target', str(stats_df['team2']['shots_on_target'])],
        [str(stats_df['team1']['passes']), 'Passes', str(stats_df['team2']['passes'])],
        [str(stats_df['team1']['pass_completion']), 'Pass Completion %', str(stats_df['team2']['pass_completion'])]
    ]
    
    return column_labels, table_vals

In [115]:
def create_xg_flow_chart(df, ax):
    df = df[df['type'] == 'Shot']
    
    #sort the values in order of time
    df = df.sort_values(by='index')
    
    #cumulative xg
    df['cumulative_xG'] = df.groupby('team_id')['shot_statsbomb_xg'].cumsum()
    
    #plot xg
    for team in df['team_id'].unique():
        team_df = df[df['team_id'] == team]
        
        team_df = pd.concat([pd.DataFrame({
            'team_id': team, 'minute': 0, 'shot_statsbomb_xg': 0, 'shot_outcome': None, 'cumulative_xG': 0, 'period': 1
        }, index=[0]), team_df])
                            
        ax.plot(
            team_df['minute'],
            team_df['cumulative_xG'],
            label=team,
            drawstyle='steps-post',
            c=ARGENTINA_COLOR if team == ARGENTINA_TEAM_ID else FRANCE_COLOR
        )
        
    for team in df['team_id'].unique():
        team_df = df[(df['team_id'] == team) & (df['shot_outcome'] == 'Goal')].to_dict(orient='records')
        for x in team_df:
            ax.scatter(
                x['minute'],
                x['cumulative_xG'],
                c='white',
                edgecolor=ARGENTINA_COLOR if team == ARGENTINA_TEAM_ID else FRANCE_COLOR,
                zorder=5
            )
            
            ax.text(
                x['minute'],
                x['cumulative_xG'] - .07,
                x['player'],
                ha='center',
                va='center',
                fontfamily='monospace',
                fontsize=8,
                zorder=10
            )
    ax.set_xticks([0, 45, 90, 120])
    ax.set_xticklabels(['0\'', '45\'', '90\'', '120\''])
    
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

In [1]:
fig = plt.figure(figsize=(15,12))

ax1 = fig.add_axes([.225, .775, .15, .15])

team1_img = Image.open('path/argentina_logo.png')
ax1.imshow(team1_img)
ax1.axis('off')

ax2 = fig.add_axes([.5, .8, .3, .1])

ax2.text(.5, .8, 'World Cup Final\nDecember 18th 2022', fontsize=12, ha='center', fontdict={'family': 'monospace'})
team_1_text= ax2.text(.47, .4, 'Argentina', fontsize=24, ha='right', fontdict={'family': 'monospace'})
team_1_text.set_bbox(dict(facecolor=ARGENTINA_COLOR, alpha=.5, edgecolor=ARGENTINA_COLOR, boxstyle='round'))

team_2_text= ax2.text(.53, .4, 'France', fontsize=24, ha='left', fontdict={'family': 'monospace'})
team_2_text.set_bbox(dict(facecolor=FRANCE_COLOR, alpha=.5, edgecolor=FRANCE_COLOR, boxstyle='round'))

ax2.text(.47, 0, '3', fontsize=20, ha='right', fontdict={'family': 'monospace'})
ax2.text(.5, 0, '-', fontsize=20, ha='center', fontdict={'family': 'monospace'})
ax2.text(.53, 0, '3', fontsize=20, ha='left', fontdict={'family': 'monospace'})

ax2.axis('off')

ax3 = fig.add_axes([.925, .775, .15, .15])

team2_img = Image.open('path/france_logo.png')
ax3.imshow(team2_img)
ax3.axis('off')

ax4 = fig.add_axes([.15, .25, .3, .5])
create_passnetwork(team1, ax4)
ax4.axis('off')

ax5 = fig.add_axes([.5, .2, .3, .5])

column_labels, table_vals = create_table(team1, team2)

table = ax5.table(
    cellText=table_vals,
    cellLoc='center',
    edges='vertical',
    bbox=[0, .5, 1, .45]
)

table.set_fontsize(14)

for (i, j), cell in table.get_celld().items():
    if j == 0:
        table.get_celld()[(i, j)].visible_edges = 'R'
    elif j == 2:
        table.get_celld()[(i,j)].visible_edges = 'L'
    else: 
        table.get_celld()[(i,j)].visible_edges = 'LR'
ax5.axis('off')

ax6 = fig.add_axes([.85, .25, .3, .5])
create_passnetwork(team2, ax6)
ax6.axis('off')

ax7 = fig.add_axes([.2, .05, .2, .2])
create_shotmap(team1, ax7)
ax7.axis('off')

ax8 = fig.add_axes([.5, .05, .3, .2])
create_xg_flow_chart(df, ax8)
ax8.axis('off')

ax9 = fig.add_axes([.9, .05, .2, .2])
create_shotmap(team2, ax9)
ax9.axis('off')

NameError: name 'plt' is not defined

In [117]:
fig.savefig('path', dpi=300, bbox_inches='tight')